<b>Exercise 6: Using external NMODL mechanisms K_Pst and Nap_Et2</b><br><br>
We continue with the four-compartment model, but in addition to the previously used Hodgkin-Huxley mechanisms we use the HH-type models for slow Na+ and K+ currents obtained from https://senselab.med.yale.edu/modeldb/showModel.cshtml?model=169457).

In [1]:
!nrnivmodl
#Import the python libraries
import matplotlib.pyplot as plt
plt.ion()
from neuron import h
import numpy
import time
from ipywidgets import widgets, fixed, Layout


/home/user/SUURPh-summer-school/L17C-18C
K_Pst.mod L4model.mod myhh.mod Nap_Et2.mod
K_Pst.mod L4model.mod myhh.mod Nap_Et2.mod
"/opt/conda/share/nrn/libtool"  --mode=compile gcc -DHAVE_CONFIG_H  -I. -I.. -I"/opt/conda/include/nrn" -I"/opt/conda/lib"      -m64 -fPIC -c -o mod_func.lo mod_func.c
libtool: compile:  gcc -DHAVE_CONFIG_H -I. -I.. -I/opt/conda/include/nrn -I/opt/conda/lib -m64 -fPIC -c mod_func.c  -fPIC -DPIC -o .libs/mod_func.o
"/opt/conda/share/nrn/libtool"  --mode=link gcc -module  -m64 -fPIC    -Wl,-rpath,/opt/conda/lib -L/opt/conda/lib -o libnrnmech.la -rpath "/opt/conda/lib"  K_Pst.lo L4model.lo myhh.lo Nap_Et2.lo mod_func.lo  -L"/opt/conda/lib" -lnrnoc -loc -lmemacs -lnrnmpi -lscopmath -lsparse13 -lreadline -lcurses -L"/opt/conda/lib" "/opt/conda/lib/libnrniv.la" -livoc -lneuron_gnu -lmeschach -lsundials -livos      -lm -ldl
libtool: link: rm -fr  .libs/libnrnmech.la .libs/libnrnmech.lai .libs/libnrnmech.so .libs/libnrnmech.so.0 .libs/libnrnmech.so.0.0.0
libtool: link:

In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
###### Run NEURON with Python interface
def runneuron(isadaptive=False,amp=0.12,stimcomp=0,stimx=0.5,glsoma=0.03,gldend=0.03,gnasoma=120,gksoma=36,gnapsoma=1.7,gkpsoma=2.2,Ra=100,cm=1.0):
    my_start_time = time.time()
        
    h.load_file("stdlib.hoc")                        #A default NEURON library
    h.load_file("stdrun.hoc")                        #A default NEURON library

    h('objref cvode')                                #Define an object cvode
    h('cvode = new CVode()')                         #Make cvode a time step integrator object
    h('cvode.active('+str(isadaptive*1)+')')         #Set the variable time step integration on/off

    h('create soma, dend[3]')                        #Create soma and three dendritic sections

    h.dend[0].connect(h.soma,1,0)                    #Connect the sections to each other
    h.dend[1].connect(h.soma,1,0)
    h.dend[2].connect(h.dend[0],1,0)
    stimtextdict = {0: h.soma, 1: h.dend[0], 2: h.dend[1], 3: h.dend[2]}
    stimnamedict = {0: 'soma', 1: 'dend[0]', 2: 'dend[1]', 3: 'dend[2]'}

    h('access soma')                                 #Make soma the currently accessed section

    for sec in [h.dend[0], h.dend[1], h.dend[2]]:
        sec.insert('pas')                            #Insert passive membrane mechanism to all sections
        sec.nseg = 5                                 #and use 5 sub-compartments for each section
    h.soma.insert('myhh')                            #Insert the Hodgkin-Huxley mechanism (includes leak) to soma
    h.soma.insert('Nap_Et2')                         #Insert persistent Na+ current
    h.soma.insert('K_Pst')                           #Insert persistent K+ current
    h.soma.nseg = 5                                  

    #Set the parameters for each section:
    h.soma.Ra = Ra; h.soma.cm = cm; h.soma.diam = 10; h.soma.L = 20; h.soma.gl_myhh = 0.001*glsoma; h.soma.gnabar_myhh = 0.001*gnasoma; h.soma.gkbar_myhh = 0.001*gksoma;
    h.soma.gNap_Et2bar_Nap_Et2 = 0.001*gnapsoma; h.soma.gK_Pstbar_K_Pst = 0.001*gkpsoma;
    h.soma.offm_K_Pst = -50; h.soma.slom_K_Pst = 3;
    h.soma.offh_K_Pst = -45; h.soma.sloh_K_Pst = 3;
    h.soma.taummax_Nap_Et2=6.0*10
    h.soma.tauhmax_Nap_Et2=1.0*0.03
    h.soma.taummin_K_Pst=1.25*3; h.soma.taumdiff1_K_Pst=175.03*3; h.soma.taumdiff2_K_Pst=13*3
    h.dend[0].Ra = Ra; h.dend[0].cm = cm; h.dend[0].diam = 5; h.dend[0].L = 200; h.dend[0].e_pas = -54.3; h.dend[0].g_pas = 0.001*gldend
    h.dend[1].Ra = Ra; h.dend[1].cm = cm; h.dend[1].diam = 2; h.dend[1].L = 100; h.dend[1].e_pas = -54.3; h.dend[1].g_pas = 0.001*gldend
    h.dend[2].Ra = Ra; h.dend[2].cm = cm; h.dend[2].diam = 2; h.dend[2].L = 160; h.dend[2].e_pas = -54.3; h.dend[2].g_pas = 0.001*gldend

    h.dt = 0.025                                     #Set the time step to 0.025 ms
    h.tstop = 200                                    #Continue the simulation until 200 ms

    h('objref stim')                                 #Declare objects stim and Vrec

    #Make stim an IClamp object, stimulating at a given location of a given compartment
    h.stim = h.IClamp(stimx, sec = stimtextdict[stimcomp])
    
    h.stim.delay = 100                               #Stimulation starts at 100 ms
    h.stim.dur = 10                                  #and lasts 10 ms
    h.stim.amp = amp                                 #and has a given amplitude (default 0.075 nA)

    h('objref mnarec, hnarec, mkrec, mnaprec, hnaprec, mkprec, hkprec, Vrec, trec')
    h('Vrec = new Vector()')
    h('trec = new Vector()')
    h('mnarec = new Vector()')
    h('hnarec = new Vector()')
    h('mkrec = new Vector()')
    h('mnaprec = new Vector()')
    h('hnaprec = new Vector()')
    h('mkprec = new Vector()')
    h('hkprec = new Vector()')
    h('soma cvode.record(&v(0.5),Vrec,trec)')
    h('soma cvode.record(&m_myhh(0.5), mnarec, trec)')
    h('soma cvode.record(&h_myhh(0.5), hnarec, trec)')
    h('soma cvode.record(&n_myhh(0.5), mkrec, trec)')
    h('soma cvode.record(&m_Nap_Et2(0.5), mnaprec, trec)')
    h('soma cvode.record(&h_Nap_Et2(0.5), hnaprec, trec)')
    h('soma cvode.record(&m_K_Pst(0.5), mkprec, trec)')
    h('soma cvode.record(&h_K_Pst(0.5), hkprec, trec)')
    
    h.init()                                         #Initialize state variables
    h.run()                                          #Run the simulations

    plt.close("all")
    f, axs = plt.subplots(2, 3, sharey=False)
    axs[0,0].plot(h.trec,h.Vrec)
    axs[0,0].set_title('$V_m$')
    axs[0,1].plot(h.trec,numpy.array(h.mnarec))
    axs[0,1].plot(h.trec,h.hnarec)
    axs[0,1].set_title('Fast Na+ m & h')
    axs[0,2].plot(h.trec,h.mkrec)
    axs[0,2].set_title('Fast K+ m')
    axs[1,1].plot(h.trec,h.mnaprec)
    axs[1,1].plot(h.trec,h.hnaprec)
    axs[1,1].set_title('Slow Na+ m & h')
    axs[1,2].plot(h.trec,h.mkprec)
    axs[1,2].plot(h.trec,h.hkprec)
    axs[1,2].set_title('Slow K+ m & h')
    for j in range(0,3):
        for i in range(0,2):
            axs[i,j].set_xlim([0,200])
            axs[i,j].set_ylim([0,1])
        axs[0,j].set_xticklabels([])
    axs[0,2].set_yticklabels([])
    axs[1,2].set_yticklabels([])
    axs[0,0].set_ylim([-80, 40])
    axs[1,0].set_visible(0)
            
    plt.suptitle("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    plt.show()


In [5]:
slider = widgets.interact(runneuron, isadaptive=fixed(True),
                          amp=fixed(0.12), stimcomp=fixed(0), stimx=fixed(0.5),description='Amplitude of square pulse current: amp',layout=Layout(width='50%'), 
                          glsoma=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, soma: glsoma',layout=Layout(width='50%'), continuous_update = False), 
                          gldend=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, dend: gldend',layout=Layout(width='50%'), continuous_update = False), 
                          gnasoma=widgets.FloatSlider(min=0.0,max=240,step=10,value=120,description='Fast Na+ conductance, soma: gnasoma',layout=Layout(width='50%'), continuous_update = False), 
                          gksoma=widgets.FloatSlider(min=0.0,max=72,step=2,value=36,description='Fast K+ conductance, soma: gksoma',layout=Layout(width='50%'), continuous_update = False), 
                          gnapsoma=widgets.FloatSlider(min=0.0,max=5,step=0.1,value=0,description='Slow Na+ conductance, soma: gnapsoma',layout=Layout(width='50%'), continuous_update = False), 
                          gkpsoma=widgets.FloatSlider(min=0.0,max=5,step=0.1,value=0,description='Slow K+ conductance, soma: gkpsoma',layout=Layout(width='50%'), continuous_update = False), 
                          Ra=fixed(100), cm=fixed(1.0))


interactive(children=(FloatSlider(value=0.03, continuous_update=False, description='Leak conductance, soma: gl…

The added mechanisms Nap_Et2 and K_Pst represent slowly activating and inactivating Na+ and K+ currents, respectively. The left-hand panel shows the somatic membrane potential, the other upper-row panels show the activation (blue) and inactivation (orange) variables of the previously used default HH Na+ (middle) and K+ (right) channels. The lower-row panels show the activation (blue) and inactivation (orange) variables for the new ion channels, Nap_Et2 (middle) and K_Pst (right). Certain parameters of the added channels have been modified in the above code to highlight their differences from the default HH channels. As in some of the previous examples, the neuron is stimulated with a square-pulse current at the soma.

#Question 1: Keeping the slow active conductances zero, study the activation and inactivation (when present) variables of all four currents. How are the kinetics of the new channels (lower panels) different from the default HH channels (upper panels)?

#Question 2: Keeping the slow K+ conductance zero, find values for the Na+ conductances and the fast K+ conductance that make the neuron produce persistent firing (or resonating around a depolarized membrane potential value) as a response to the stimulus at $t=100$ ms. Now, compare again the dynamics of the activation and inactivation variables of the persistent Na+ and K+ currents to those of the fast currents. Can you find another difference in addition to those you described in question 1?

#***Question 3***: Adding the slow K+ conductance, find values for all the active conductances that make the neuron fire multiple action potentials as a response to the stimulus at $t=100$ ms and hyperpolarize afterwards.